# FMM Recbole 구현

### Recbole 라이브러리 로딩

In [84]:
# !pip install recbole

In [85]:
# !pip install ray

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

from logging import getLogger
import os
import json
import numpy as np
import pandas as pd
import time, datetime
from tqdm import tqdm

from recbole.model.context_aware_recommender.ffm import FFM

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.data.interaction import Interaction
from recbole.utils import init_logger, get_trainer, init_seed, set_color, get_model
from recbole.quick_start.quick_start import load_data_and_model

from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import ndcg_score, recall_score

import torch

### 데이터 로드

In [3]:
train_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 118.0 MB


In [5]:
data_path = '/opt/ml/input/data/train'
year_data = pd.read_csv(os.path.join(data_path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(data_path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(data_path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(data_path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(data_path, 'directors.tsv'), sep='\t')

In [6]:
df_merge = pd.merge(train_df, year_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, writer_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, title_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, genre_data.drop_duplicates(subset=['item']), on='item', how='inner')
df_merge = pd.merge(df_merge, director_data.drop_duplicates(subset=['item']), on='item', how='inner')

In [8]:
df_merge = pd.read_csv('./uitgyt.csv')
director_data = pd.read_csv('./director_fe.csv')
writer_data = pd.read_csv('./writer_fe.csv')
df_merge = pd.merge(df_merge, director_data, on='item', how='left')
df_merge = pd.merge(df_merge, writer_data, on='item', how='left')

In [82]:
df_merge = df_merge.sort_values('user')

In [83]:
df_merge.head()

,user,item,time,genre_embedding,year,title,director_info,writer_info
0,11,4643,1230782529,-0.009958,2001.0,Planet of the Apes (2001),2859.444444,4146.333333
255,11,8640,1230856739,-0.010216,2004.0,King Arthur (2004),704.666667,8310.000000
254,11,8907,1230856729,-0.010698,2004.0,Shark Tale (2004),9518.666667,28810.000000
253,11,8965,1230856675,-0.010338,2004.0,"Polar Express, The (2004)",3960.875000,5677.444444
252,11,36401,1230856670,-0.010524,2005.0,"Brothers Grimm, The (2005)",4461.615385,1339.923077


In [84]:
train_data = df_merge[['user', 'item', 'time']]

In [85]:
user_data = df_merge[['user']]

In [86]:
item_data = df_merge[['item', 'year', 'writer_info', 'title', 'genre_embedding', 'director_info']]#.drop_duplicates(subset=['item'])

In [87]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 157.3 MB


In [88]:
userid, itemid = list(set(train_data.user)), list(set(train_data.item))
n_user, n_item = len(userid), len(itemid)
print(f'n_user : {n_user}')
print(f'n_item : {n_item}')

n_user : 31360
n_item : 6807


### 데이터 전처리

### 데이터 파일 변환

기존 데이터 파일을 Recbole 데이터 파일로 변환시키는 과정

In [89]:
userid, itemid = sorted(userid), sorted(itemid)
n_user, n_item = len(userid), len(itemid)

userid_2_index = {v:i for i,v in enumerate(userid)}
itemid_2_index = {v:i for i,v in enumerate(itemid)}
index_2_userid = {i:v for i,v in enumerate(userid)}
index_2_itemid = {i:v for i,v in enumerate(itemid)}

In [90]:
yamldata = """
field_separator: "\t"
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, timestamp]
    user: [user_id]
    item: [item_id, year, writer, title, genre, director]

train_neg_sample_args:
    uniform: 1
    
eval_args:
    split: {'RS': [4, 1, 1]}
    group_by: user
    order: RO
    mode: full
metrics: ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk: 10
valid_metric: Recall@10
"""

In [91]:
train_data.user = train_data.user.map(userid_2_index)
train_data.item = train_data.item.map(itemid_2_index)

user_data.user = user_data.user.map(userid_2_index)
item_data.item = item_data.item.map(itemid_2_index)

/tmp/ipykernel_27941/1297655130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.user = train_data.user.map(userid_2_index)
/tmp/ipykernel_27941/1297655130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.item = train_data.item.map(itemid_2_index)
/tmp/ipykernel_27941/1297655130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [92]:
train_data.columns=['user_id:token', 'item_id:token', 'timestamp:float']
user_data.columns=['user_id:token']
item_data.columns=['item_id:token', 'year:token', 'writer:token', 'title:token_seq', 'genre:token', 'director:token']


In [93]:
outpath = f"dataset/train_data"
# outfile = f"dataset/train_data/train_data.inter"
yamlfile = f"train_data.yaml"

os.makedirs(outpath, exist_ok=True)
# sub_train=train.groupby("user").sample(n=10, random_state=SEED)
# sub_train.shape

# print("Processing Start")
# inter_table = []
# for user, item, time in zip(train_data.user, train_data.item, train_data.time):
#     uid, iid = userid_2_index[user], itemid_2_index[item]
#     # tval = int(time.mktime(datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S").timetuple()))
#     inter_table.append( [uid, iid, time] )

# print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
train_data.to_csv(os.path.join(outpath,"train_data.inter"),sep='\t',index=False)
user_data.to_csv(os.path.join(outpath,"train_data.user"),sep='\t',index=False)
item_data.to_csv(os.path.join(outpath,"train_data.item"),sep='\t',index=False)
# with open(outfile, "w") as f:
#     # write header
#     f.write("user_id:token\titem_id:token\ttimestamp:float\n")
#     for row in inter_table:
#         f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Dump Start


447

Dump Complete


### 로거 생성

In [94]:
logger = getLogger()

### 설정 인스턴스 생성

In [95]:
# configurations initialization
config = Config(model='FFM', dataset="train_data", config_file_list=[f'train_data.yaml'])
config['epochs'] = 1
config['show_progress'] = False
config['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

logger.info(config)

26 Dec 14:10    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_se

### 데이터 로드

In [96]:
# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

26 Dec 14:12    INFO  train_data
The number of users: 31361
Average actions of users: 164.36450892857144
The number of items: 6808
Average actions of items: 757.2309387395328
The number of inters: 5154471
The sparsity of the dataset: 97.58579218741939%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director']
26 Dec 14:12    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
26 Dec 14:12    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [97]:
train_data.dataset
valid_data.dataset
test_data.dataset

train_data
The number of users: 31361
Average actions of users: 110.4156568877551
The number of items: 6808
Average actions of items: 508.68738063757894
The number of inters: 3462635
The sparsity of the dataset: 98.37820011614866%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

train_data
The number of users: 31361
Average actions of users: 26.974426020408163
The number of items: 6808
Average actions of items: 124.29003820158684
The number of inters: 845918
The sparsity of the dataset: 99.60379603563536%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

train_data
The number of users: 31361
Average actions of users: 26.974426020408163
The number of items: 6808
Average actions of items: 124.27177905097693
The number of inters: 845918
The sparsity of the dataset: 99.60379603563536%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'year', 'writer', 'title', 'genre', 'director', 'label']

### 모델 인스턴스 생성

In [98]:
# model loading and initialization
init_seed(config['seed'], config['reproducibility'])
model = FFM(config, train_data.dataset).to(config['device'])
logger.info(model)

26 Dec 14:12    INFO  FFM(
  (token_embedding_table): FMEmbedding(
    (embedding): Embedding(42227, 10)
  )
  (token_seq_embedding_table): ModuleList(
    (0): Embedding(9166, 10)
  )
  (first_order_linear): FMFirstOrderLinear(
    (token_embedding_table): FMEmbedding(
      (embedding): Embedding(42227, 1)
    )
    (token_seq_embedding_table): ModuleList(
      (0): Embedding(9166, 1)
    )
  )
  (sigmoid): Sigmoid()
  (ffm): FieldAwareFactorizationMachine(
    (token_embeddings): ModuleList(
      (0): Embedding(42227, 10)
      (1): Embedding(42227, 10)
      (2): Embedding(42227, 10)
      (3): Embedding(42227, 10)
      (4): Embedding(42227, 10)
      (5): Embedding(42227, 10)
      (6): Embedding(42227, 10)
    )
    (token_seq_embeddings): ModuleList(
      (0): ModuleList(
        (0): Embedding(9166, 10)
        (1): Embedding(9166, 10)
        (2): Embedding(9166, 10)
        (3): Embedding(9166, 10)
        (4): Embedding(9166, 10)
        (5): Embedding(9166, 10)
        

### 모델 학습

In [99]:
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)

26 Dec 14:14    INFO  epoch 0 training [time: 74.21s, train loss: 1392.1670]
26 Dec 14:19    INFO  epoch 0 evaluating [time: 290.06s, valid_score: 0.079700]
26 Dec 14:19    INFO  valid result: 
recall@10 : 0.0797    mrr@10 : 0.3824    ndcg@10 : 0.1852    hit@10 : 0.7579    precision@10 : 0.1727    map@10 : 0.0876
26 Dec 14:19    INFO  Saving current: saved/FFM-Dec-26-2022_14-12-59.pth


### 학습 결과 출력

In [100]:
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model="True", show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

result = {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

print(json.dumps(result, indent=4))

26 Dec 14:19    INFO  Loading model structure and parameters from saved/FFM-Dec-26-2022_14-12-59.pth
26 Dec 14:23    INFO  best valid : OrderedDict([('recall@10', 0.0797), ('mrr@10', 0.3824), ('ndcg@10', 0.1852), ('hit@10', 0.7579), ('precision@10', 0.1727), ('map@10', 0.0876)])
26 Dec 14:23    INFO  test result: OrderedDict([('recall@10', 0.0943), ('mrr@10', 0.4578), ('ndcg@10', 0.2372), ('hit@10', 0.7858), ('precision@10', 0.2159), ('map@10', 0.13)])


{
    "best_valid_score": 0.0797,
    "valid_score_bigger": true,
    "best_valid_result": {
        "recall@10": 0.0797,
        "mrr@10": 0.3824,
        "ndcg@10": 0.1852,
        "hit@10": 0.7579,
        "precision@10": 0.1727,
        "map@10": 0.0876
    },
    "test_result": {
        "recall@10": 0.0943,
        "mrr@10": 0.4578,
        "ndcg@10": 0.2372,
        "hit@10": 0.7858,
        "precision@10": 0.2159,
        "map@10": 0.13
    }
}


In [101]:
# 모델 불러오기
model_path='saved/FFM-Dec-26-2022_14-12-59.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_path)

26 Dec 14:23    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [4, 1, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_se

## Inference

In [102]:
from recbole.utils.case_study import full_sort_topk

user_id = config['USER_ID_FIELD']
user_id2token = dataset.field2id_token[user_id]
all_user_list = torch.arange(1, len(user_id2token)).view(-1,128)

device = config.final_config_dict['device']

tbar = tqdm(all_user_list, desc=set_color(f"Inference", 'pink'), leave=True, mininterval=1)

pred_list = None
user_list = []
for data in tbar:
    batch_pred_list = full_sort_topk(data, model, test_data, 10, device=device)[1]
    batch_pred_list = batch_pred_list.clone().detach().cpu().numpy()
    if pred_list is None:
        pred_list = batch_pred_list
        user_list = data.numpy()
    else:
        pred_list = np.append(pred_list, batch_pred_list, axis=0)
        user_list = np.append(
            user_list, data.numpy(), axis=0
        )
tbar.close()


Inference:   0%|          | 0/245 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)
Inference:   5%|▌         | 13/245 [00:01<00:19, 12.20it/s]:  11%|█         | 26/245 [00:02<00:17, 12.22it/s]:  16%|█▌        | 39/245 [00:03<00:16, 12.23it/s]:  21%|██        | 52/245 [00:04<00:15, 12.24it/s]:  27%|██▋       | 65/245 [00:05<00:14, 12.21it/s]:  32%|███▏      | 78/245 [00:06<00:13, 12.18it/s]:  37%|███▋      | 91/245 [00:07<00:12, 12.18it/s]:  42%|████▏     | 104/245 [00:08<00:11, 12.20it/s]:  48%|████▊     | 117/245 [00:09<00:10, 12.21it/s]:  53%|█████▎    | 130/245 [00:10<00:09, 12.20it/s]:  58%|█████▊    | 143/245 [00:11<00:08, 12.20it/s]:  64%|██████▎   | 156/245 [00:12<00:07, 12.20it/s]:  69%|██████▉   | 169/245 [0

array([ 106,  154,  297,  370,  354,  648,  358,  279,  412, 1406])

In [103]:
# user별 item 추천 결과 하나로 합쳐주기
result = []
for user, pred in zip(user_list, pred_list):
    item_cnt = 0 # 10개가 되면 다음 유저로 이동
    user = int(index_2_userid[user-1])
    user_before = df_merge[df_merge['user'] == user]['item'].unique()
    for item in pred:
        item = int(index_2_itemid[item-1])
        if item_cnt >= 10:
            continue
        if item not in user_before:
            result.append((user, item))
            item_cnt += 1

In [104]:
# submission file 제작하기
sub = pd.DataFrame(result, columns=["user", "item"])
sub.to_csv(
    "submission_ffm.csv", index=False
)
print('inference done!')

inference done!
